In [3]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score
from statsmodels.stats.weightstats import *

from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

## Критерии Стьюдента
#### Тест 1

In [4]:
n = 160
mu0 = 9.5
x_mean = 9.57
s = 0.4
def z_1sample(x_mean, mu0, s, n):
    return (x_mean - mu0) / (s / np.sqrt(n))

In [5]:
#так как известно стандартное отклонение из ген совокупности, то исполь-ся Z-критерий!
2 * (1 - stats.norm.cdf(abs(z_1sample(x_mean, mu0, s, n))))

0.026856695507523787

In [6]:
df = pd.read_csv('diamonds.txt', sep='\t')

In [7]:
df.head(2)

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['price'], axis=1), df['price'],
                                                    test_size=0.25, random_state=1)

In [9]:
lin_mod = LinearRegression()
lin_mod.fit(X_train, y_train)
y_pred_lin = lin_mod.predict(X_test)

rfc = RandomForestRegressor(random_state=1)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)

/Users/Barnett/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [10]:
#просто для себя посчитал метрику
mean_absolute_error(y_test, y_pred_lin), mean_absolute_error(y_test, y_pred_rfc)

(890.3764004285592, 802.9205172724115)

In [11]:
abs_error_lin = abs(y_test - y_pred_lin)
abs_error_rfc = abs(y_test - y_pred_rfc)

In [12]:
#так как на одних и тех же даных => выборки связанные
stats.ttest_rel(abs_error_lin, abs_error_rfc)

Ttest_relResult(statistic=13.017729783878393, pvalue=1.655174575144972e-38)

In [13]:
print("95%% confidence interval: [%f, %f]" % DescrStatsW(abs_error_lin - abs_error_rfc).tconfint_mean())

95% confidence interval: [74.287245, 100.624521]


## Параметрические тесты
#### Тест 2

In [14]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)
    
def proportions_diff_z_stat_ind(n1=None, n2=None, cnt_pos1=None, cnt_pos2=None,
                                p1=None, p2=None, sample1=None, sample2=None):
    if sample1 is not None:
        n1 = len(sample1)
        cnt_pos1 = sum(sample1)
        #подсчет вероятноти для 1-й популяции
        p1 = cnt_pos1 / n1
    elif n1 is not None and cnt_pos1 is not None:
        p1 = cnt_pos1 / n1
    else:
        assert(n1 is not None) , 'You give no sample, no cnt_pos1, so it must be p1 and n1'
        
    if sample2 is not None:
        n2 = len(sample2)
        cnt_pos2 = sum(sample2)
        #подсчет вероятноти для 2-й популяции
        p2 = cnt_pos2 / n2
    elif n2 is not None and cnt_pos2 is not None:
        p2 = cnt_pos2 / n2
    else:
        assert(n1 is not None) , 'You give no sample, no cnt_pos1, so it must be p1 and n1'
        
    P = (p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return (f - g) / np.sqrt(f + g - (f - g)**2 / n)

In [16]:
#3
#группа поделена на эксп и контр => выборки независимые => используем Z-критерий для доли для незав выборок

n1, n2 = 34, 16
k_pos1, k_pos2 = 10, 4
print(100*abs(k_pos1 / n1 - k_pos2 / n2), '%')
z_stat = proportions_diff_z_stat_ind(n1=n1, n2=n2, cnt_pos1=k_pos1, cnt_pos2=k_pos2)
print(proportions_diff_z_test(z_stat, alternative='greater'))

4.411764705882354 %
0.37293045872523534


In [145]:
#4 выборки связанные, тк обучаемся на одних и тех же объектах!!
#несмотря на то, что признаки объектов берем разные, можно грубо считать это как параметр модели-выбор признаков!!
df = pd.read_csv('banknotes.txt', sep='\t')
X_train, X_test, y_train, y_test = train_test_split(df.drop(['real'], axis=1), df['real'], random_state=1,
                                                    test_size=50)
print('Check length: {0}'.format(X_test.shape[0]==50))

Check length: True


In [209]:
logreg1 = LogisticRegression(solver='liblinear')
logreg2 = LogisticRegression(solver='liblinear')
logreg1.fit(X_train[['X1', 'X2', 'X3']], y_train)
logreg2.fit(X_train[['X4', 'X5', 'X6']], y_train)
y_pr1 = logreg1.predict(X_test[['X1', 'X2', 'X3']])
y_pr2 = logreg2.predict(X_test[['X4', 'X5', 'X6']])

z_stat = proportions_diff_z_stat_rel(y_test!=y_pr1, y_test!=y_pr2)
p_value = proportions_diff_z_test(z_stat)
print(p_value, z_stat)
print('Accuracy: 1-st clf: {0}, 2-d clf: {1}'.format(accuracy_score(y_test, y_pr1),accuracy_score(y_test, y_pr2)))


0.0032969384555543435 2.9386041680175268
Accuracy: 1-st clf: 0.8, 2-d clf: 0.98


In [211]:
#5
proportions_diff_confint_rel(y_test!=y_pr1, y_test!=y_pr2)

(0.059945206279614305, 0.3000547937203857)

In [219]:
#6
z_stat = (541.4-525)/(100/np.sqrt(100))
print(z_stat)
proportions_diff_z_test(z_stat, 'greater')

1.6399999999999977


0.05050258347410397

In [220]:
#7
z_stat = (541.5-525)/(100/np.sqrt(100))
print(z_stat)
proportions_diff_z_test(z_stat, 'greater')

1.65


0.0494714680336481

## Непараметрические критерии
#### Тест 3

In [305]:
#4
print("M: %d, p-value: %f" % stats.wilcoxon(np.array([49,58,75,110,112,132,151,276,281,362]) - 200))

M: 17, p-value: 0.284503


In [329]:
#так получается результат который вычисляется в лекциях, используя знаковый критерий и альтернативу "больше"
l = [49,58,75,110,112,132,151,276,281,362]
cnt_1 = sum([1 if x > 200 else 0 for x in l])
print(stats.binom_test(cnt_1, len(l), 0.5, alternative = 'greater'))
#а если руками, то получается так!
bindist = stats.binom(10, 0.5)
pmf = bindist.pmf(range(10))
sum([x for x, ix in zip(pmf, range(10)) if ix >= 3])

0.9453125


0.9443359375000009

In [301]:
#5
stats.mannwhitneyu([22,22,15,13,19,19,18,20,21,13,13,15], [17,18,18,15,12,4,14])

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

In [287]:
#6    
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [265]:
df = pd.read_csv('challenger.txt', sep='\t', index_col=0)
df.head()

,Temperature,Incident
Apr12.81,18.9,0
Nov12.81,21.1,1
Mar22.82,20.6,0
Nov11.82,20.0,0
Apr04.83,19.4,0


In [284]:
temp0 = df[df['Incident']==0]['Temperature'].values
temp1 = df[df['Incident']==1]['Temperature'].values

In [275]:
np.random.seed(0)
samples0 = get_bootstrap_samples(temp0, 1000)
samples1 = get_bootstrap_samples(temp1, 1000)

In [291]:
samples0_mean = np.array(list(map(np.mean, samples0)))
samples1_mean = np.array(list(map(np.mean, samples1)))
stat_intervals(samples0_mean-samples1_mean,0.05)

array([1.42299107, 7.93861607])

In [294]:
#7
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), list(filter(lambda i: i not in index, range(n)))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [299]:
np.random.seed(0)
print("p-value: %f" % permutation_test(temp0, temp1, max_permutations = 10000))

p-value: 0.007000
